In [1]:
import os
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')
import torch  
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.transforms as transforms 
import numpy as np
from sklearn import *
import csv

In [3]:
# Pls refer to https://pytorch.org/docs/stable/data.html
class EE4146_Dataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, file_path):
        """
        Args:
            csv_path (string): csv 
            img_path (string): 
            transform: transform 
        """
        self.file_path = file_path
        self.to_tensor = transforms.ToTensor() 
        self.data_info = pd.read_csv(csv_path, header=0) 
        self.image_arr = np.asarray(data_info.iloc[:, 0])
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        self.data_len = len(self.data_info.index)
        self.encode_labels = dict( zip(np.unique(self.label_arr), range(len(np.unique(self.label_arr)))))
        print('encoded labels: ', self.encode_labels)
        self.transform = torchvision.transforms.Compose([
                    transforms.Resize((150,150)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.425, 0.415, 0.405), (0.255, 0.245, 0.235))
                    ])

    def __getitem__(self, index):
        img_id = self.image_arr[index]
        img_as_img = Image.open(self.file_path + str(img_id) + ".jpg")
        img_as_img = self.transform(img_as_img)
        label = self.encode_labels[self.label_arr[index]]
        return (img_as_img, label, img_id)  
        
    def __len__(self):
        return self.data_len

In [ ]:
# Define pretrained deep feature extractor
model = torchvision.models.wide_resnet50_2(pretrained=True).cuda()
for param in model.parameters():
    param.required_grad = False

# Replacing the final classifier by identity layer (for directly out features)
model.fc = torch.nn.Identity()
model.eval()


In [7]:
# Define dataloaded
train_data = EE4146_Dataset('train_labels.csv','raw_images/train/' )
train_loader = torch.utils.data.DataLoader(train_data,batch_size=8,num_workers=0, shuffle=False)

encoded labels:  {'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}


In [ ]:
# Inference and save features
label_save = []
feats_save = []
feat_save_path = 'train_feat/'

if not os.path.exists(feat_save_path):
    os.makedirs(feat_save_path)

for images, labels, image_ids in train_loader:
    B, C, W, H = images.shape
    feat = model(images.cuda()).detach().cpu()
    feats_save.append(feat) # B 2024
    label_save.append(labels)
    for i, imgID in enumerate(image_ids):
        np.save(feat_save_path +'{}_feat_res50.npy'.format(imgID), feat[i].numpy())

feats_save = torch.cat(feats_save,dim=0)
label_save = torch.cat(label_save)
